In [1]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Engine in Vertex AI - Multiple tools




### Install Vertex AI SDK for Python

In [2]:
%pip install --upgrade --user --quiet google-cloud-aiplatform[agent_engines,langchain,ag2]

In [3]:
# Note:  Need to restart the kernel
%pip install --upgrade --user --quiet langchain-google-vertexai

In [4]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
#MODEL_NAME = "gemini-1.5-flash-002"
MODEL_NAME = "gemini-2.0-flash-001"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

In [5]:
# @title Create a bucket.
BUCKET_URI = f"gs://agent-0319"
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0319/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0319' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [6]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")

SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [7]:
# @title Set access to the bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://agent-0319/
No changes made to gs://agent-0319/


In [8]:
# @title Initialize Vertex AI with Staging Bucket.

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Tools definition

In [9]:
# Define Search tool

from vertexai.generative_models import grounding, Tool

grounded_search_tool = Tool.from_google_search_retrieval(
    grounding.GoogleSearchRetrieval()
)

In [10]:
# Get capital city
def get_capital_return(
    country:str, ):

  """Retrieves the capital city name of the given country"""

  from vertexai.generative_models import (
      GenerationConfig,
      GenerativeModel,
      HarmBlockThreshold,
      HarmCategory,
      Part,
  )

  model = GenerativeModel(MODEL_NAME)
  prompt = f"Answer the capital city of {country}"

  responses = model.generate_content(
        prompt,
  )

  return responses.text

In [11]:
# Get exchange rate
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

### Agent - Local unit test

In [12]:
# from vertexai.preview.generative_models import ToolConfig
from vertexai.preview.reasoning_engines import LangchainAgent

# Agent Define
agent = LangchainAgent(
    model=MODEL_NAME,
    tools=[get_exchange_rate,
           get_capital_return,
    ],

    agent_executor_kwargs={"return_intermediate_steps": False},
)

#### Test your agent locally


In [13]:
# Agent local unit test before deploying.
agent.query(input="what's the capital city of south korea")

{'input': "what's the capital city of south korea",
 'output': 'The capital city of South Korea is Seoul.\n'}

In [14]:
# Agent local unit test before deploying.
agent.query(input="What's the exchange rate from US dollars to EUR currency today?")

{'input': "What's the exchange rate from US dollars to EUR currency today?",
 'output': 'Could you please specify the exact date you want me to check?\n'}